<a href="https://colab.research.google.com/github/OSegun/Data-Engineering-Projects/blob/main/Production_DB_Building_a_database_for_crime_reports.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building a database for crime reports

In [ ]:
import psycopg2

conn = psycopg2.connect(dbname = 'dq', user = 'dq')
conn.autocommit = True
cur = conn.cursor()
cur.execute("CREATE DATABASE crime_db")
conn.autocommit = False
conn.close()

In [ ]:
new_conn = psycopg2.connect(dbname = 'crime_db', user = 'dq')
cur = new_conn.cursor()
cur.execute("CREATE SCHEMA crimes")
new_conn.commit()

In [ ]:
import csv

with open('boston.csv') as file:
    reader = list(csv.reader(file))
    col_headers = reader[0]
    first_row = reader[1]
    
print(col_headers)
print("\n")
print(first_row)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']


['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


In [ ]:
'''def get_col_set(csv_filename, col_index):
    col_set = set()
    with open(csv_filename) as file:
        next(file)
        reader = list(csv.reader(file))
        for val in reader:
            value = val[col_index]
            col_set.add(value)
        return col_set
        
        
get_col_set("boston.csv", 2) '''

'def get_col_set(csv_filename, col_index):\n    col_set = set()\n    with open(csv_filename) as file:\n        next(file)\n        reader = list(csv.reader(file))\n        for val in reader:\n            value = val[col_index]\n            col_set.add(value)\n        return col_set\n        \n        \nget_col_set("boston.csv", 2) '

In [ ]:
def get_col_set(csv_file, col_index):
    import csv
    values = set()
    with open(csv_file, 'r') as f:
        next(f)
        reader = csv.reader(f)
        for row in reader:
            values.add(row[col_index])
    return values

for i in range(len(col_headers)):
    values = get_col_set("boston.csv", i)
    #print(col_headers[i], len(values), sep='\t')

In [ ]:
print(col_headers)
print("\n")
print(first_row)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']


['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


In [ ]:
get_col_set("boston.csv", 5)

{'42.30998781',
 '42.3639917',
 '42.31965582',
 '42.33311205',
 '42.3247019',
 '42.28870802',
 '42.28560634',
 '42.31525538',
 '42.33078637',
 '42.32757423',
 '42.33860282',
 '42.25428304',
 '42.36129655',
 '42.27631028',
 '42.30097791',
 '42.28267661',
 '42.34008002',
 '42.36418457',
 '42.27853182',
 '42.37321072',
 '42.35932797',
 '42.25027603',
 '42.27776448',
 '42.28113011',
 '42.35133919',
 '42.37640973',
 '42.34175616',
 '42.3214866',
 '42.32818615',
 '42.26128526',
 '42.3663835',
 '42.32712014',
 '42.27189172',
 '42.34914832',
 '42.29057318',
 '42.30295884',
 '42.27342041',
 '42.31948219',
 '42.35706352',
 '42.30046668',
 '42.30555133',
 '42.28090773',
 '42.27064206',
 '42.3205644',
 '42.2930945',
 '42.29125987',
 '42.28677022',
 '42.34925003',
 '42.33030529',
 '42.29073974',
 '42.28771066',
 '42.28547822',
 '42.31358713',
 '42.31106474',
 '42.35912459',
 '42.28538033',
 '42.33097596',
 '42.28929213',
 '42.27582518',
 '42.25980165',
 '42.30209842',
 '42.28674125',
 '42.31490192'

In [ ]:
get_col_set("boston.csv", 4)

{'Friday', 'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday'}

In [ ]:
description = get_col_set("boston.csv", 2)
max_len = 0
for val in description:
    max_len = max(max_len, len(val))
print(max_len)


58


In [ ]:
new_conn = psycopg2.connect(dbname = 'crime_db', user = 'dq')
cur = new_conn.cursor()
cur.execute("""CREATE TYPE day AS ENUM 
                    ('Friday', 'Monday', 'Saturday', 
                    'Sunday', 'Thursday', 'Tuesday', 
                    'Wednesday');""")
new_conn.commit()

In [ ]:
new_conn = psycopg2.connect(dbname = 'crime_db', user = 'dq')
cur = new_conn.cursor()
cur.execute("""CREATE TABLE crimes.boston_crimes 
                    (incident_number INTEGER PRIMARY KEY, 
                    offense_code INTEGER, 
                    description varchar(100), 
                    date DATE, 
                    week_day day, 
                    latitude decimal,
                    logitude decimal);""")

In [ ]:
with open("boston.csv", 'r') as f:
    cur.copy_expert("COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER;", f)

In [ ]:
cur.execute("""REVOKE ALL ON SCHEMA public FROM public;""")

cur.execute("""REVOKE ALL ON DATABASE crime_db FROM public;""")

In [ ]:
cur.execute("""CREATE GROUP readonly NOLOGIN;""")

cur.execute("""CREATE GROUP readwrite NOLOGIN;""")

ProgrammingError: role "readonly" already exists


In [ ]:
cur.execute("""GRANT INSERT, SELECT, DELETE, UPDATE ON ALL TABLES IN SCHEMA crimes TO readwrite;""")

cur.execute("""GRANT CONNECT ON DATABASE crime_db TO readwrite;""")

cur.execute("""GRANT USAGE ON SCHEMA crimes TO readwrite;""")

InternalError: current transaction is aborted, commands ignored until end of transaction block


In [ ]:
cur.execute("""GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly;""")

cur.execute("""GRANT CONNECT ON DATABASE crime_db TO readonly;""")

cur.execute("""GRANT USAGE ON SCHEMA crimes TO readonly;""")

InternalError: current transaction is aborted, commands ignored until end of transaction block


In [ ]:
new_conn.commit()

In [ ]:
cur.execute("""REVOKE ALL ON SCHEMA public FROM public;""")

cur.execute("""REVOKE ALL ON DATABASE crime_db FROM public;""")

In [ ]:
cur.execute("""CREATE GROUP readonly NOLOGIN;""")

cur.execute("""CREATE GROUP readwrite NOLOGIN;""")

InternalError: current transaction is aborted, commands ignored until end of transaction block
